**<font color = black size=6>实验六:随机森林</font>**

In [478]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
from collections import Counter

**<font color = blue size=4>第一部分:函数介绍</font>**

介绍一些可能会用到的函数。

In [479]:
# np.random.choice函数从一个一维数组中随机采样
x = np.array([1,2,3,4])
y = np.random.choice(x, replace=True, size=10)
print(y)

# np.random.shuffle函数对一个数组/矩阵按照第一维进行洗牌
x = np.array([[0,1,2],[3,4,5],[6,7,8],[9,10,11],[12,13,14]])
np.random.shuffle(x)
print(x)

# DataFrame对象的sample函数可以随机采样n个数据或者采样比例为frac的数据
x = np.array([[0,0,0],[1,1,1],[2,2,2],[3,3,3],[4,4,4]])
frame = pd.DataFrame(x)
print(frame.sample(n=2))
print(frame.sample(frac=0.3))

[1 3 4 1 4 1 4 1 1 1]
[[12 13 14]
 [ 9 10 11]
 [ 6  7  8]
 [ 3  4  5]
 [ 0  1  2]]
   0  1  2
4  4  4  4
0  0  0  0
   0  1  2
2  2  2  2
0  0  0  0


**<font color = blue size=4>第二部分:实验任务</font>**

本次实验承接上次实验，实现随机森林。

<span style="color:purple">本次实验依旧使用泰坦尼克号数据集(train_titanic.csv, test_titanic.csv。数据集包括了四个属性特征以及一个标签(即为Survived,代表是否生还),属性特征分别为Sex(性别)，sibsp(堂兄弟妹个数)，Parch(父母与小孩的个数)，Pclass(乘客等级)  
其中该数据集无缺失值和异常值，且所有连续变量已自动转换为离散变量,标签(Survived)也自动转变为离散变量0和1，所以你无需进行数据预处理，可以直接使用该数据集。</span>

<span style="color:purple">1) 对上次实验的best_split函数进行修改，实现随机特征选择。  
先从特征集$A$中先随机选取$k$个特征构成特征集$A'$，再从$A'$中选取最佳划分的特征。$k$一般取$max\{log_2 d,1\}$, $d$是$A$的元素的个数。你可使用特征的信息增益来决定最佳划分的特征。  
    【输入】：数据集D、特征集A    
    【输出】：随机特征集A'中最佳划分的特征维数   
    【信息增益公式】:  
        某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,这里取其中一个特征feature,该特征包含V个不同的取值，特征值为第v(v=1,2,...,V)个值的数量为|$D^v$|$(\sum_{v=1}^VD^v=|D|)$,则该特征对应的信息增益为$$Gain(D,feature)=Ent(D)-\sum_{v=1}^K \frac{|D^v|}{D} Ent(D^v)$$  
    【信息熵公式】:  
        某数组包含K个不同的取值，样本为第k(k=1,2,...,K)个值的数量所占比例为p_k,则其信息熵为$$Ent=-\sum_{k=1}^K p_k log_2 p_k$$
</span>

In [480]:
def best_split(D, A):
    d = len(A) # d是A的元素的个数
    k = max(round(math.log2(d)), 1) # 计算k值
    
    # 将set A转换为list，然后进行随机采样
    A_prime = random.sample(list(A), k)  # 从A中随机选取k个属性构成属性集A'
    
    best_entropy = float('-inf')  # 初始化最佳信息增益值为无穷小
    best_dimension = -1  # 初始化最佳维度  (可省略)
    base_entropy = calculate_entropy(D[:, -1])  # 数据集D的标签是最后一列 计算基础信息熵
    
    features = D[:, :-1] # 特征值
    labels = D[:, -1] # 标签值
    # 对于A'中的每个特征
    for d in A_prime:
        # 按当前维度划分数据
        _, split_labels = split_dataset(features, labels, d) 
        # 计算特征划分后的信息熵
        new_entropy = np.sum([len(sub_labels) / len(labels) * calculate_entropy(sub_labels) for sub_labels in split_labels]) 
        # 计算信息增益值
        info_gain = base_entropy - new_entropy 
        # 若当前的信息增益值更大,则进行更新
        if info_gain > best_entropy:
            best_entropy = info_gain  # (可省略)
            best_dimension = d # 更新最佳划分特征维度
            
    # 返回最佳的特征维度
    return best_dimension 

def calculate_entropy(label):
    label = label.reshape(-1,1) # 设定形状
    total_count = len(label) # 标签的总数
    label_counts = Counter(label[:,0]) # 统计每个标签的数量
    ent = 0 # 初始化信息熵为0
    for count in label_counts.values():
        p_k = count / total_count # 计算标签的比例
        ent += -p_k * math.log2(p_k) # 累加信息熵
    return ent

def split_dataset(feature, label, d):    
    # 获取该维度的所有不重复的值
    unique_values = np.unique(feature[:, d])

    # 初始化空的列表来保存划分后的数据
    split_feature = []
    split_label = []

    # 根据unique_values的值，划分feature和label
    for value in unique_values:
        # 对于等于 unique_value 的子集
        matched_indices = feature[:, d] == value # 匹配则对应位置的索引为True
        # 根据匹配值来确定是否加入相应的矩阵行
        subset_split_feature = feature[matched_indices]
        subset_split_label = label[matched_indices]
        # 添加结果
        split_feature.append(subset_split_feature)
        split_label.append(subset_split_label)
    # 转换为矩阵
    split_feature = np.array(split_feature,dtype=object)
    split_label = np.array(split_label,dtype=object)
    # 返回划分结果
    return split_feature, split_label

<span style="color:purple">2) 对上次实验完成的决策树类进行修改。你需要实现下面三个函数：  
1. TreeGenerate(self, D, A)：递归构建决策树，伪代码参照提供的“Algorithm 1 决策树学习基本算法”。  
2. train(self, D)：做一些数据预处理，包括将Dataframe转换为numpy矩阵，从数据集中提取属性集，并调用TreeGenerate函数来递归地生成决策树。  
3. predict(self, D)：对测试集D进行预测，要求返回数据集D的预测标签，即一个(|D|,1)矩阵（|D|行1列）。  
由于训练集是采样生成，因此需要对predict函数做修改。需要考虑测试集中出现决策树无法划分的特征值时的情况。给出两种参考的做法：  
a).对其不再进行预测，直接给定划分失败的样本标签(例如-1)。  
b).跳过该划分节点，随机选取一个特征值继续遍历。</span>

In [481]:
# 记下所有属性可能的取值
train_frame = pd.read_csv('train_titanic.csv')
D = np.array(train_frame)
A = set(range(D.shape[1] - 1))
possible_value = {}
for every in A:
    possible_value[every] = np.unique(D[:, every])

In [482]:
# 树结点类
class Node:
    def __init__(self, isLeaf=True, label=-1, index=-1):
        self.isLeaf = isLeaf # isLeaf表示该结点是否是叶结点
        self.label = label # label表示该叶结点的label（当结点为叶结点时有用）
        self.index = index # index表示该分支结点的划分属性的序号（当结点为分支结点时有用）
        self.children = {} # children表示该结点的所有孩子结点，dict类型，方便进行决策树的搜索
        
    def addNode(self, val, node):
        self.children[val] = node #为当前结点增加一个划分属性的值为val的孩子结点

In [483]:
# 决策树类
class DTree:
    def __init__(self):
        self.tree_root = None #决策树的根结点
        self.possible_value = possible_value # 用于存储每个属性可能的取值
    
        
    '''
    TreeGenerate函数用于递归构建决策树,伪代码参照课件中的"Algorithm 1 决策树学习基本算法"
    '''
    def TreeGenerate(self, D, A):
        
        # 生成结点 node
        node = Node()
        
        
        
        # if D中样本全属于同一类别C then
        #     将node标记为C类叶结点并返回
        # end if
        
        unique_labels = np.unique(D[:, -1]) # 获取非重复的样本值
        # 所有样本同一类别
        if len(unique_labels) == 1:
            node.isLeaf = True # 标记为叶子节点
            node.label = unique_labels[0] # 标记label
            return node
        
        
        # if A = Ø OR D中样本在A上取值相同 then
        #     将node标记叶结点，其类别标记为D中样本数最多的类并返回
        # end if
        
        if len(A) == 0 or np.all([len(np.unique(D[:, feature])) == 1 for feature in A]):
            node.isLeaf = True  # 标记为叶子结点
            node.label = Counter(D[:, -1]).most_common(1)[0][0] # 标记label为D中样本数最多的类
            return node
        
        
        # 从A中选择最优划分属性a_star
        # （选择信息增益最大的属性，用到上面实现的best_split函数）
        # a_star = best_split(D, A)
        a_star = best_split(D,A) # 选择信息增益最大的特征
        node.index = a_star
        node.isLeaf = False # 标记为非叶子结点
        
        
        # for a_star 的每一个值a_star_v do
        #     为node 生成每一个分支；令D_v表示D中在a_star上取值为a_star_v的样本子集
        #     if D_v 为空 then
        #         将分支结点标记为叶结点，其类别标记为D中样本最多的类
        #     else
        #         以TreeGenerate(D_v,A-{a_star}) 为分支结点
        #     end if
        # end for

        for a_star_v in self.possible_value[a_star]:
            child_node = Node() # 初始化分支结点
            D_v = D[D[:, a_star] == a_star_v] # 生成D_v
            
            if len(D_v) == 0:
                child_node.isLeaf = True # 标记为叶子结点
                child_node.label = Counter(D[:,-1]).most_common(1)[0][0] # 标记类别为D中样本最多的类
            else:
                child_node = self.TreeGenerate(D_v, A - {a_star})
            # 添加分支结点
            node.addNode(a_star_v, child_node)   
        
        return node
    
    
    
    
    '''
    train函数可以做一些数据预处理(比如Dataframe到numpy矩阵的转换,提取属性集等),并调用TreeGenerate函数来递归地生成决策树
    '''
    def train(self, D):
#         D = np.array(D) # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）
#         A = set(range(D.shape[1] - 1)) # 属性集A
#         self.tree_root = self.TreeGenerate(D, A) # 递归地生成决策树，并将决策树的根结点赋值给self.tree_root
        D = np.array(D) 
        A = set(range(D.shape[1] - 1)) 
        self.tree_root = self.TreeGenerate(D, A) 
        return 
    
    
    '''
    predict函数对测试集D进行预测,输出预测标签
    '''
    def predict(self, D):
#         D = np.array(D) # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）
        
#         #对于D中的每一行数据d，从当前结点x=self.tree_root开始，当当前结点x为分支结点时，
#         #则搜索x的划分属性为该行数据相应的属性值的孩子结点（即x=x.children[d[x.index]]），不断重复，
#         #直至搜索到叶结点，该叶结点的label就是数据d的预测label
        D = np.array(D) # 转换为numpy矩阵
        predictions = [] # 数据集D的预测标签列表
        # 遍历每一行
        for i in range(D.shape[0]):
            x = self.tree_root # 从当前节点x=self.tree_root开始
            d = D[i] # D中每一行数据d
            # 直至搜索到叶子结点
            while not x.isLeaf:
                # 考虑测试集中出现决策树无法划分的特征值
                if d[x.index] not in x.children:
                    # 若采用方法b 跳过该划分结点 随机选取一个特征值继续遍历
                    children_keys = list(x.children.keys())
                    random_key = np.random.choice(children_keys)
                    x = x.children[random_key] 
                    # 若采用方法a  则直接将-1添加到预测标签列表中
                    # predictions.append(-1)   
                    # break
                else:
                    x=x.children[d[x.index]]
            # if predictions[-1] != -1:    # 如果采用方法a 则需要去掉该行注释 
            predictions.append(x.label) # 添加该预测标签
        return np.array(predictions).reshape(-1,1) # 转换为numpy矩阵(|D|,1)

<span style="color:purple">3) 重复采用Bootstrap自助采样法对训练数据集'test_titanic.csv'进行采样，生成$n$个子训练数据集($n$自行设定)。  
Bootstrap采样法是指，每次从原数据集中【有放回】地随机采样一个样本，重复进行$m$次，就生成一个有$m$个样本的子数据集。</span>

In [484]:
train_frame = pd.read_csv('train_titanic.csv')

# Bootstrap 采样

def bootstrap_sampling(data, n):
    # 对数据进行自助采样 生成n个训练数据集 输入的data格式为dataframe
    samples = [] # 用于存放n个采样得到的数据集
    m = len(data) # 原始数据集的数据个数m
    # 生成n个训练数据集
    for _ in range(n):
        sample = data.sample(m, replace=True) # 有放回地采样m个数据
        samples.append(sample) # 加入该训练数据集
    # 返回包含n个训练数据集的列表
    return samples

# 设定n的值
n = 40 
# 调用函数 得到采样的结果
train_samples = bootstrap_sampling(train_frame,n)
print(train_samples)


[     Sex  sibsp  Parch  Pclass  Survived
318    1      0      2       1         1
24     1      3      1       3         0
543    0      0      0       3         0
539    0      0      0       1         1
943    0      0      0       2         0
..   ...    ...    ...     ...       ...
351    0      0      0       1         0
907    0      1      0       1         0
703    1      0      0       1         0
495    0      0      0       2         0
918    0      0      0       1         0

[1009 rows x 5 columns],      Sex  sibsp  Parch  Pclass  Survived
771    0      0      0       2         0
836    0      1      0       1         0
27     0      3      2       1         0
359    1      0      0       3         1
285    0      0      0       3         0
..   ...    ...    ...     ...       ...
214    0      1      0       3         0
515    0      0      0       1         0
652    0      0      0       2         0
851    0      1      0       3         0
340    0      1      1       2

<span style="color:purple">4) 生成n棵决策树实例，使用上述生成的n个子训练数据集各自训练一棵决策树，即子训练集D1训练决策树1，子训练集D2训练决策树2……</span>

In [485]:
# ----- Your code here -------


trees = [] # 用于存放训练好的决策树

# 创建n棵决策树实例并对每个数据集进行训练
for i in range(n):
    dtree = DTree()  # 创建一个新的决策树实例
    dtree.train(train_samples[i])  # 使用第i个训练数据集训练决策树
    trees.append(dtree)  # 将训练好的决策树添加到森林中

<span style="color:purple">5) 用训练完成的$n$棵决策树分别对测试数据集'test_titanic.csv'进行预测。采用相对多数投票法来对各棵决策树的预测结果进行结合。输出结合的预测结果的准确率。  
【相对多数投票法】  
对于某个样本$x$, 相对多数投票法预测它的标签为得票最多的标签。若同时有多个标签获得最高票，则从中随机选取一个。其公式如下所示：
$$H(x)=C_{\mathop{\arg\max}_{j} \sum_{i=1}^n h_i^j(x)}$$  
</span>

In [486]:
test_frame = pd.read_csv('test_titanic.csv')

# ----- Your code here -------
test_data = np.array(test_frame) # 格式转换

X_test = test_data[:,:-1] # 测试集特征
y_true = test_data[:,-1].reshape(-1,1) # 测试集标签


# 使用n棵决策树对测试数据集进行预测
all_predictions = []
for tree in trees:
    # 调用函数进行在测试集特征上 获取预测结果
    predictions = tree.predict(X_test) 
    # 存储测试集特征的预测结果
    all_predictions.append(predictions)
    

# 使用相对多数投票法进行预测结果的结合
final_predictions = []

for i in range(y_true.shape[0]):
    # 获取每个样本在n棵决策树中的预测结果 存储为votes列表
    votes = [predictions[i][0] for predictions in all_predictions] # 注意predictions的形状为(|D|,1)
    # 使用Counter来统计每个预测结果的出现次数
    vote_count = Counter(votes)
    # 获取最高票数
    max_votes = vote_count.most_common(1)[0][1]
    # 创建一个包含所有获得最高票数的标签的列表
    top_votes_labels = [label for label, count in vote_count.items() if count == max_votes]
    # 从中随机选择一个标签
    most_common_prediction = random.choice(top_votes_labels)
    # 将选中的预测结果进行添加
    final_predictions.append(most_common_prediction)

final_predictions = np.array(final_predictions).reshape(-1,1) # 进行形状转换 便于与t_true直接进行比较

# 计算预测准确率
accuracy = np.mean(final_predictions == y_true)
print(f"结合的预测结果的准确率为: {accuracy}")

结合的预测结果的准确率为: 0.8316831683168316


**<font color = blue size=4>第三部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)不要提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可

二、实验报告提交截止日期为：【10月27日 14:20】  
提交地址：https://send2me.cn/g_kfMtFI/SuiqyPO6B7rxqg  
要求：  
1)文件格式为：学号-姓名-实验六.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  

三、课堂课件获取地址: https://www.jianguoyun.com/p/DTGgCYAQp5WhChir26EFIAA  
实验内容获取地址: https://www.jianguoyun.com/p/DekWAFoQp5WhChis26EFIAA